In [51]:
# Imports
import pandas as pd

In [36]:
# Reading data from source and creating df
storm_df = pd.read_csv('data/events-US-1980-2021.csv',
                       header=1,
                       parse_dates=['Begin Date', 'End Date'])

In [37]:
storm_df.dtypes

Name                                                     object
Disaster                                                 object
Begin Date                                       datetime64[ns]
End Date                                         datetime64[ns]
Total CPI-Adjusted Cost (Millions of Dollars)           float64
Deaths                                                    int64
dtype: object

In [44]:
storm_df['Disaster'].value_counts()

Disaster
Severe Storm        152
Tropical Cyclone     57
Flooding             36
Drought              29
Winter Storm         20
Wildfire             20
Freeze                9
Name: count, dtype: int64

In [54]:
# Filtering the dataset to match the original visualisation
severe_storm_df = storm_df.loc[storm_df['Disaster'] == 'Severe Storm'].copy(deep=True)

In [55]:
severe_storm_df

,Name,Disaster,Begin Date,End Date,Total CPI-Adjusted Cost (Millions of Dollars),Deaths
4,"Severe Storms, Flash Floods, Hail, Tornadoes (...",Severe Storm,1981-05-05,1981-05-10,1268.6,20
6,Midwest/Plains/Southeast Tornadoes (April 1982),Severe Storm,1982-04-02,1982-04-04,1444.8,33
7,Severe Storms (June 1982),Severe Storm,1982-05-31,1982-06-10,1420.6,30
13,"Tornadoes, Severe Storms, Floods (Spring 1984)",Severe Storm,1984-03-27,1984-04-07,1670.5,80
14,Severe Storms and Hail (June 1984),Severe Storm,1984-06-13,1984-06-17,1222.6,1
...,...,...,...,...,...,...
312,Central Severe Storms (June 2021),Severe Storm,2021-06-24,2021-06-26,1303.0,0
314,Central Severe Storms (July 2021),Severe Storm,2021-07-08,2021-07-11,1123.2,0
315,North Central Severe Weather (August 2021),Severe Storm,2021-08-10,2021-08-13,1335.3,2
319,"Southeast, Central Tornado Outbreak (December ...",Severe Storm,2021-12-10,2021-12-10,3993.3,93


In [ ]:
# TODO add seasons columns (2 types), month and year columns 